In [3]:
import os
import cv2
import numpy as np
import tensorflow as tf
import sys

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

In [4]:
from utils import label_map_util
from utils import visualization_utils as vis_util

In [5]:
MODEL_NAME = 'C:/Users/Ankita/Desktop/models-master/research/object_detection/inference_graph'
IMAGE_NAME = 'C:/Users/Ankita/Desktop/models-master/research/object_detection/images/train/salman_khan_test.jpg'

In [6]:
CWD_PATH = os.getcwd()
CWD_PATH

'C:\\Users\\Ankita\\Desktop\\models-master\\research\\object_detection'

In [7]:
PATH_TO_CKPT = os.path.join(CWD_PATH,MODEL_NAME,'frozen_inference_graph.pb')
PATH_TO_CKPT

'C:/Users/Ankita/Desktop/models-master/research/object_detection/inference\\frozen_inference_graph.pb'

In [8]:
PATH_TO_LABELS = os.path.join(CWD_PATH,'training','labelmap.pbtxt')
PATH_TO_LABELS

'C:\\Users\\Ankita\\Desktop\\models-master\\research\\object_detection\\training\\labelmap.pbtxt'

In [9]:
PATH_TO_IMAGE = os.path.join(CWD_PATH,IMAGE_NAME)

In [10]:
NUM_CLASSES = 2

In [11]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [12]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    sess = tf.Session(graph=detection_graph)
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
num_detections = detection_graph.get_tensor_by_name('num_detections:0')
image = cv2.imread(PATH_TO_IMAGE)
image = cv2.resize(image,(800,600))
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

image_expanded = np.expand_dims(image_rgb, axis=0)
(boxes, scores, classes, num) = sess.run(
    [detection_boxes, detection_scores, detection_classes, num_detections],
    feed_dict={image_tensor: image_expanded})
im=vis_util.visualize_boxes_and_labels_on_image_array(
    image,
    np.squeeze(boxes),
    np.squeeze(classes).astype(np.int32),
    np.squeeze(scores),
    category_index,
    use_normalized_coordinates=True,
    line_thickness=8,
    min_score_thresh=0.60)

In [13]:
ymin, xmin, ymax, xmax =vis_util.box_dim(image,
    np.squeeze(boxes),
    np.squeeze(classes).astype(np.int32),
    np.squeeze(scores),
    category_index,
    use_normalized_coordinates=True,
    line_thickness=8,
    min_score_thresh=0.60)

In [15]:
image_height, image_width, abd =image.shape
x_min=int(xmin * image_width)
y_min=int(ymin * image_height)
x_max=int(xmax * image_width)
y_max=int(ymax * image_height)
cropped_img = image[y_min:y_max,x_min:x_max,:]
cv2.imshow('Object detector', cropped_img)

#image

In [16]:
cv2.waitKey(0)

-1

In [45]:
filename = 'images/train/sunglasses_extracted.jpg'
  
# Using cv2.imwrite() method 
# Saving the image 
cv2.imwrite(filename, cropped_img) 

True

In [43]:
cv2.imshow('Object detector', image)
cv2.waitKey(0)
filename = 'images/train/sunglasses_detected.jpg'
  
# Using cv2.imwrite() method 
# Saving the image 
cv2.imwrite(filename, image) 

True

In [134]:
cv2.destroyAllWindows()